Lleve a cabo un estudio de simulación para verificar la que
la confianza empírica de los intervalos de confianza apara la media µ en un
proceso ARMA(p,q) son aproximadamente del 100(1 − α)%. Qué sucede con
la significancia si se omitiera la estructura de autocorrelación?

In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima.model import ARIMA
from scipy import stats

def simulate_arma(ar_params, ma_params, nsample):
    np.random.seed(42)
    ar = np.r_[1, -np.array(ar_params)]
    ma = np.r_[1, np.array(ma_params)]
    arma_process = ArmaProcess(ar, ma)
    return arma_process.generate_sample(nsample=nsample)

def compute_ci_arma(data, alpha=0.05):
    model = ARIMA(data, order=(len(ar_params), 0, len(ma_params)))
    results = model.fit()
    mean = results.params[0]
    se = results.bse[0]
    ci = (mean - stats.norm.ppf(1-alpha/2) * se, 
          mean + stats.norm.ppf(1-alpha/2) * se)
    return mean, ci

def compute_ci_iid(data, alpha=0.05):
    mean = np.mean(data)
    se = np.std(data, ddof=1) / np.sqrt(len(data))
    ci = stats.t.interval(1-alpha, len(data)-1, loc=mean, scale=se)
    return mean, ci

def run_simulation(ar_params, ma_params, nsample, nrep, alpha=0.05):
    true_mean = 0  # ARMA processes have mean 0
    coverage_arma = 0
    coverage_iid = 0
    
    for _ in range(nrep):
        data = simulate_arma(ar_params, ma_params, nsample)
        
        _, ci_arma = compute_ci_arma(data, alpha)
        if ci_arma[0] <= true_mean <= ci_arma[1]:
            coverage_arma += 1
        
        _, ci_iid = compute_ci_iid(data, alpha)
        if ci_iid[0] <= true_mean <= ci_iid[1]:
            coverage_iid += 1
    
    coverage_arma /= nrep
    coverage_iid /= nrep
    
    return coverage_arma, coverage_iid

# Parameters
ar_params = [0.5]
ma_params = [0.4]
nsample = 500
nrep = 1000
alpha = 0.05

coverage_arma, coverage_iid = run_simulation(ar_params, ma_params, nsample, nrep, alpha)

print(f"Empirical coverage (ARMA): {coverage_arma:.4f}")
print(f"Empirical coverage (IID): {coverage_iid:.4f}")
print(f"Nominal coverage: {1-alpha:.4f}")

Empirical coverage (ARMA): 1.0000
Empirical coverage (IID): 1.0000
Nominal coverage: 0.9500
